# Funnel Queries (funnel models)

In [ ]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop **product funnel** queries with funnel-specific models.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
min_date = '2021-02-09'
max_date = '2021-02-12'

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Queries

### Using Intermediate Model

In [ ]:
%%sql
/* ============= MARTS fct_sessions_daily DATA MODEL STARTS HERE ============= */
WITH users AS (
    SELECT user_id,
           address_id
    FROM stg_postgres_users
),
addresses AS (
    SELECT address_id,
           state AS state_name
    FROM stg_postgres_addresses
),
products AS (
    SELECT product_id,
           name AS product_name
    FROM stg_postgres_products
),
event_users AS (
    SELECT DISTINCT(user_id) AS user_id
    FROM stg_postgres_events
),
daily_user_sessions AS (
    SELECT *
    FROM int_sessions_aggregated_to_product_daily
),
user_addresses AS (
     SELECT u.user_id,
            a.state_name
     FROM event_users AS u
     INNER JOIN users us USING (user_id)
     INNER JOIN addresses a USING (address_id)
),
daily_user_sessions_named AS (
    SELECT s.session_id,
           s.user_id,
           -- event_id,
           ua.state_name,
           p.product_name,
           s.created_at_date,
           ZEROIFNULL(s.num_page_views) AS num_page_views,
           ZEROIFNULL(s.num_add_to_carts) AS num_add_to_carts,
           ZEROIFNULL(s.num_checkouts) AS num_checkouts
    FROM daily_user_sessions AS s
    INNER JOIN user_addresses ua USING (user_id)
    INNER JOIN products p USING (product_id)
),
/* ============= MARTS fct_sessions_daily DATA MODEL ENDS HERE ============= */
/* ============= BI TOOL STARTS HERE ============= */
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
overall AS (
    SELECT ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT COUNT(DISTINCT(session_id)) AS page_views,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
    ) t1
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS add_to_carts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
    ) t2 USING (row_num)
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS checkouts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
    ) t3 USING (row_num)
    ORDER BY checkouts DESC
),
by_state AS (
    SELECT state_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name)
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name)
    ORDER BY checkouts DESC
),
by_product AS (
    SELECT product_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name)
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name)
    ORDER BY checkouts DESC
),
overall_daily AS (
    SELECT created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (created_at_date)
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
by_state_daily AS (
    SELECT state_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name, created_at_date)
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name, created_at_date)
    ORDER BY state_name, created_at_date, checkouts DESC
),
by_product_daily AS (
    SELECT product_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name, created_at_date)
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name, created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
overall_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM overall
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_state
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_product
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
overall_daily_pivotted AS (
    SELECT created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM overall_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_daily_pivotted AS (
    SELECT state_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_state_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_daily_pivotted AS (
    SELECT product_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_product_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
)
/* ============= BI TOOL ENDS HERE ============= */
SELECT *
FROM overall_daily

### Using Intermediate and Marts Models

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
/* ============= BI TOOL STARTS HERE ============= */
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
overall AS (
    SELECT ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT COUNT(DISTINCT(session_id)) AS page_views,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
    ) t1
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS add_to_carts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
    ) t2 USING (row_num)
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS checkouts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
    ) t3 USING (row_num)
    ORDER BY checkouts DESC
),
by_state AS (
    SELECT state_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name)
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name)
    ORDER BY checkouts DESC
),
by_product AS (
    SELECT product_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name)
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name)
    ORDER BY checkouts DESC
),
overall_daily AS (
    SELECT created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (created_at_date)
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
by_state_daily AS (
    SELECT state_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name, created_at_date)
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name, created_at_date)
    ORDER BY state_name, created_at_date, checkouts DESC
),
by_product_daily AS (
    SELECT product_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name, created_at_date)
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name, created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
overall_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM overall
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_state
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_product
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
overall_daily_pivotted AS (
    SELECT created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM overall_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_daily_pivotted AS (
    SELECT state_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_state_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_daily_pivotted AS (
    SELECT product_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_product_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
)
/* ============= BI TOOL ENDS HERE ============= */
SELECT *
FROM overall_daily

## BI Tool Queries

### Overall

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
overall AS (
    SELECT ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT COUNT(DISTINCT(session_id)) AS page_views,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
    ) t1
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS add_to_carts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
    ) t2 USING (row_num)
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS checkouts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
    ) t3 USING (row_num)
    ORDER BY checkouts DESC
)
SELECT *
FROM overall

### By State

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
by_state AS (
    SELECT state_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name)
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name)
    ORDER BY checkouts DESC
)
SELECT *
FROM by_state

### By Product

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
by_product AS (
    SELECT product_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name)
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name)
    ORDER BY checkouts DESC
)
SELECT *
FROM by_product

### Overall Daily

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
overall_daily AS (
    SELECT created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (created_at_date)
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (created_at_date)
    ORDER BY created_at_date, checkouts DESC
)
SELECT *
FROM overall_daily

### By State Daily

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
by_state_daily AS (
    SELECT state_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name, created_at_date)
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name, created_at_date)
    ORDER BY state_name, created_at_date, checkouts DESC
)
SELECT *
FROM by_state_daily

### By Product Daily

In [ ]:
%%sql
WITH daily_user_sessions_named AS (
    SELECT *
    FROM fct_sessions_daily
),
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
by_product_daily AS (
    SELECT product_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name, created_at_date)
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name, created_at_date)
    ORDER BY created_at_date, checkouts DESC
)
SELECT *
FROM by_product_daily

## Disconnect

Close connection

In [ ]:
%sql --close connection